#Objective:

Using K-means and Complete Hierarchical Single i can klustering the pokemon data base

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

#Importación de las librerías de interés
import numpy as np                # linear algebra
import pandas as pd               # data frames
import seaborn as sns             # visualizations
import matplotlib.pyplot as plt   # visualizations
import scipy.stats                # statistics

from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import complete, fcluster #Cluster Jerarquico Completo

from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
#Se renombra la base de datos a trabajar y se realiza la exploración inicial
df = pd.read_csv("../input/pokemon6/pokemon6.csv")

# Print the head of df
print(df.head())
# Print the info of df
print(df.info())
# Print the shape of df
print(df.shape)

In [ ]:
It looks like we have information for 801 Pokemons across 32 variables.

#Basic Exploratory Data Analysis

In [ ]:
#Médidas básicas para todas las variables
df.describe()

In [ ]:
#¿La base está balaceada?
df.isnull().sum()

In [ ]:
df['height_m'].fillna(0,inplace = True)
df['percentage_male'].fillna(0,inplace = True)
df['weight_kg'].fillna(0,inplace = True)
df.isnull().sum()

In [ ]:
#Médidas básicas para todas las variables
df.describe()

In [ ]:
#histograma para variables continuas
f, axes = plt.subplots(3,2, figsize=(20, 12))
f.suptitle("Distribución variables continuas", fontsize=20)
sns.distplot( df["attack"], ax=axes[0,0])
sns.distplot( df["defense"], ax=axes[0,1])
sns.distplot( df["hp"], ax=axes[1,1])
sns.distplot( df["speed"], ax=axes[2,0])
sns.distplot( df["sp_defense"], ax=axes[2,1])

In [ ]:
#Histogramas para las variables categoricas 
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(18, 20))
fig.suptitle("Distribución de frecuencias para las variables categoricas", fontsize=20)

x = ['Legendary','Non Legendary'];y = df.is_legendary.value_counts(sort = False).values
axes[0][0].bar(x,y);axes[0][0].set_title('Antiguedad')

x = ['Muy Debil','Debil','Promedio','fuerte','poderoso'];y = df.against_bug.value_counts(sort = False).values
axes[0][1].bar(x,y);axes[0][1].set_title('Contra Insectos')

x = ['Muy Debil','Debil','Promedio','fuerte','poderoso'];y = df.against_dark.value_counts(sort = False).values
axes[0][2].bar(x,y);axes[0][2].set_title('Contra Oscuridad')

x = ['Muy Debil','Debil','Promedio','fuerte'];y = df.against_dragon.value_counts(sort = False).values
axes[1][0].bar(x,y);axes[1][0].set_title('Contra Dragones')

x = ['Muy Debil','Debil','Promedio','fuerte','Muy Fuerte','Poderoso'];y = df.against_electric.value_counts(sort = False).values
axes[1][1].bar(x,y);axes[1][1].set_title('Contra Electricos')

x = ['Muy Debil','Debil','Promedio','fuerte','poderoso'];y = df.against_fairy.value_counts(sort = False).values
axes[1][2].bar(x,y);axes[1][2].set_title('Contra Hadas')
plt.show()

#We can Check for example that the strong or more ist a mesuare not efective, has a lettle information in comparisson like weik measures

In [ ]:
# Compute the correlation matrix
corr=df.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool);mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, center=0,square=True, linewidths=.5, cbar_kws={"shrink": .5})

#Data Transformations

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 10))
fig.suptitle("base Total Vs. Attack & Defense", fontsize=20)

sns.catplot(x="attack", y="defense", hue="is_legendary",kind="point", ax=axes[0],data=df )
sns.catplot(x="hp", y="speed", hue="is_legendary",kind="point", data=df,ax=axes[1])

In [ ]:
#Codificando las variables categoricas
df = pd.read_csv("../input/pokemon6/pokemon6.csv")
#creamos variables categoricas separadas y quitamos una para no generar problemas de multicolinealidad
df_dum=pd.get_dummies(df,columns=["against_bug","against_dark","against_dragon","against_electric","against_fairy","against_fight","against_fire","against_flying","against_ghost","against_grass","against_ground","against_ice","against_normal","against_poison","against_psychic","against_rock","against_steel","against_water","attack","defense","experience_growth","height_m","hp","percentage_male","pokedex_number","sp_attack","sp_defense","speed","weight_kg","generation","is_legendary"],drop_first=True)
df_dum.describe()

In [ ]:
pokemon_scale = df_dum
scaler = preprocessing.StandardScaler()
columns = df_dum.columns
pokemon_scale[columns] = scaler.fit_transform(pokemon_scale[columns])
pokemon_scale.head()

In [ ]:
Z = linkage(pokemon_scale.loc[:,["base_total","is_legendary_1"]], 'complete', metric='euclidean')
fig = plt.figure(figsize=(25, 10))
dn = dendrogram(Z)
plt.show()

In [ ]:
h_cluster=df.copy()
h_cluster['5_clust']=fcluster(Z,t=5, criterion='distance')
h_cluster['3_clust']=fcluster(Z,t=3, criterion='distance')
h_cluster.head()

In [ ]:
# Compute the correlation matrix
corr_cluster=h_cluster.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr_cluster, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr_cluster, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

We use 5 clusters for correlation:

Los Clusters

Cluster 1 = Experiance

Cluster 2 = Power

Cluster 3 = Defense

cluster 4 = Generation

Cluster 5 = Hierarchy

#Hierarchical clustering / Single

In [ ]:
Z = linkage(pokemon_scale.loc[:,["base_total","generation_2","is_legendary_1"]],
            'single', metric='euclidean')
fig = plt.figure(figsize=(25, 10))
dn = dendrogram(Z)
plt.show()

In [ ]:
h_single=df.copy()
h_single['7_clust']=fcluster(Z,t=2, criterion='distance')

h_single.head()

In [ ]:
# Compute the correlation matrix
corr_single=h_single.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr_single, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr_single, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
We use 2 clusters for correlation:

Los Clusters

Cluster 1 = Experiance

Cluster 2 = Power